In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer as WNL
import scipy
from IPython.display import clear_output as clr
%matplotlib inline

In [2]:
# Defining data directory
data_dir = os.getcwd() + '/../'
# Check in contents of current dir are same as expected
os.listdir(data_dir)

['.ipynb_checkpoints',
 '20news-18828.tar.gz',
 'corpuses',
 'LDA',
 'NMF',
 'stackoverflow-data-idf.json',
 'stackoverflow-test.json',
 'TF-IDF',
 'train_corpus',
 'Unigram']

In [3]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer as WNL

In [4]:
class preprocessor:
    
    def __init__(self):
        # word lemmatizer
        self.articles = 'a|an|and|the'
        self.lemmatizer = WNL()
        self.punctuations = string.punctuation
        self.translator = str.maketrans(string.punctuation, ' '*len(self.punctuations))
        
    def normalize_text(self, s):
        lower_s = self.lower(s)
        rem_p_s = self.remove_punc(lower_s)
        rem_a_s = self.remove_articles(rem_p_s)
        space_s = self.white_space_fix(rem_a_s)
        lemma_s = self.lemmatize(space_s)
        return lemma_s
        
    def lemmatize(self, txt):
        lemmatizer = self.lemmatizer
        return ' '.join(lemmatizer.lemmatize(lemmatizer.lemmatize(word, pos = 'v'), pos = 'a') for word in txt.split())

    def remove_articles(self, text):
        return re.sub('\s+('+self.articles+')(\s+)', ' ',text)

    def white_space_fix(self, text):
        return ' '.join(text.split())

    def remove_punc(self, text):
        return text.translate(self.translator)

    def lower(self, text):
        return text.lower()

In [5]:
processor = preprocessor()

In [6]:
from nltk.corpus import brown
 
data = []
 
for fileid in brown.fileids():
    document = ' '.join(brown.words(fileid))
    data.append(processor.normalize_text(document))
#     if(len(data) >= 5):
#         break

In [7]:
class vocab:
    
    def __init__(self, corpus):
        """
        Corpus : list of docs
        """
        self.dictionary = {}
        self.vocab_size = 0
        self.make_vocab(corpus)
        self.inv_dictionary = {v: k for k, v in self.dictionary.items()}
    
    def make_vocab(self,corpus):
        
        count = 0
        for doc in corpus:
            for word in doc.strip().split():
                if(word in self.dictionary.keys()):
                    pass
                else:
                    self.dictionary[word] = count
                    count += 1
        
        self.vocab_size = count
        assert(len(self.dictionary.keys()) == count), "Wrong Number of words added"
        
    def get_index(self,word):
        if(word in self.dictionary.keys()):
            return self.dictionary[word]
        else:
            return -1
    
    def get_word(self,index):
        if(index in self.inv_dictionary.keys()):
            return self.inv_dictionary[index]
        else:
            return " "

In [8]:
myDict = vocab(data)

In [9]:
myDict.vocab_size

34110

In [13]:
def make_mat(corpus, vocab):
    
    D = len(corpus)
    V = vocab.vocab_size
    mat = np.zeros((V, D))
    for i in range(len(corpus)):
        doc = corpus[i]
        for word in doc.strip().split(" "):
            index = vocab.get_index(word)
            mat[index,i] += 1
    return mat

In [14]:
X = make_mat(data, myDict)

In [15]:
X.shape

(34110, 500)

In [38]:
def NMF(X, k, Winit = None, Hinit = None, convex = True):    
    V, D = X.shape
    eps = 0.0001
    diff = 10
    c_iter = 0
    
    if(Winit is None):
        W = np.random.rand(V, k)
        H = np.random.rand(k, D)
    else:
        W = Winit
        H = Hinit

    while(diff > eps and c_iter < 5000):
        W_old = W.copy()
        H_old = H.copy()
        c_iter+=1
    
        if(convex):
            H = H/np.sum(H, axis =1).reshape(-1,1)
        W = W*(np.dot(X,H.T)/np.dot(W,np.dot(H, H.T)))
        H = H*(np.dot(W.T,X)/np.dot(W.T,np.dot(W,H)))
        
        diff = np.sum(np.square(W_old - W))
        if(c_iter%10 == 0):
            print(diff)
    return W, H

In [ ]:
W, H = NMF(X, 20)

1130012.5605385592
217860.54016032603
135066.5391539044
64730.64282975136
34631.52166726363
21502.308777503764
14581.648298135799
10414.168803999399
6937.525900950751
5328.104165921641
4685.5776618434
3787.8798270396755
3792.9096858811436
2901.992165401409
2610.126952084029
2428.289029420683
2180.4622773980423
1988.1860628095553
2209.453955124831
1670.2594940503513
1634.662830465643
1449.0875141421252
1296.0275256930454
1206.8573053236435
1117.1957472588629
1029.2423898211382
919.4821378775505
893.9900428974242
813.188258867439
865.0617225501435
813.0274117938724
781.0220989859589
728.3905737318701
634.3020231520101
630.0437296046432
557.6937085255008
539.5448723920584
507.1956896467989
541.0610205035691
552.7820794246971
445.22518303682244
424.04834819649994
379.8967552884592
393.4945404328069
320.34105912957637
286.6886945591219
268.8907597139791
253.92286014494155
231.335279717315
223.18549961419194
218.89895282032333
205.09664860799447
212.75951965177694
179.54442260505868
368.0988

In [32]:
H.max()

3.3548064624404237